In [1]:
# Hypothesis: smaller kernel sizes will lead to higher accuracies of classification in fined-tuned image classification
# We think this is likely due to the ability of the CNN to detect smaller features between relatively similar images of the same species
# Independent Variables: kernel size
# Constant Variables: number of layers, types of layers, input shape, epoches, hyperparamaters(depth,padding)
# Dependent Variables: Accuracy
# Limitations: we are foregoing pure accuracy for experimental reasons - we could get higher accuracy if we tried but we're keeping
# things constant for consistancy between models
# Currently only using Input, Convolutional, Relu, Pool, and FC/Dense layers. Can add Dense and Dropout if we have time
# Reference to www.cs231n.github.io/convolutional-networks

In [2]:
# 3. Import libraries and modules
import os, cv2, argparse
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from numpy.random import RandomState
np.random.seed(123)  # for reproducibility
import sys
import random

/home/cfu288/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def shuffle_data(arr1, arr2):
    seed = random.randint(0, 1000)
    ran = RandomState(seed)
    ran.shuffle(arr1)
    ran = RandomState(seed)
    ran.shuffle(arr2)

In [4]:
# INPUT
TRAIN_DIR = './training/'
TEST_DIR = './validation/'
EPOCHS = 10
kernels = [(1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]
num_filters = 210
input_shape=(200,200,3)
input_shape1=(200,200)
run_times = 4

In [5]:
# Setup, Read in images, Preprocess images for training
TRAIN_IMG, TRAIN_CLS, TEST_IMG, TEST_CLS = ([] for i in range(4))
COLS = ['Label', 'Latin Name', 'Common Name', 'Train Images', 'Validation Images']
LABELS = pd.read_csv('./monkey_labels.txt', names=COLS, skiprows=1)
CLASSES = [x for x in range(0, len(LABELS))]

# read in all images
# resizing the images to 100x100 to make training faster
print("READING IN AND FORMATTING IMAGES")
for x in range(0, len(LABELS)):
    train_dir = TRAIN_DIR + LABELS.loc[x,'Label'].strip() + '/'
    test_dir = TEST_DIR + LABELS.loc[x,'Label'].strip() + '/'
    for file in os.listdir(train_dir):
        if not file.endswith('jpg'):
            print("empty file")
            continue
        img = cv2.imread(train_dir + file)
        if img is not None:
            img = cv2.resize(img, input_shape1)
            TRAIN_IMG.append(img)
            TRAIN_CLS.append(x)
    for file in os.listdir(test_dir):
        img = cv2.imread(test_dir + file)
        if img is not None:
            img = cv2.resize(img, input_shape1)
            TEST_IMG.append(img)
            TEST_CLS.append(x)
print("COMPLETE")

READING IN AND FORMATTING IMAGES
empty file
COMPLETE


In [6]:
# convert to numpy arrays
TRAIN_IMG = np.array(TRAIN_IMG)
TEST_IMG = np.array(TEST_IMG)
TRAIN_CLS = np.array(TRAIN_CLS)
TEST_CLS = np.array(TEST_CLS)

# Preprocess images
# Reshape them to theanos format (channels, hight, width) # changed to tensorflow
# Convert to 0-255 to value in [0-1]
# TRAIN_IMG = TRAIN_IMG.reshape(TRAIN_IMG.shape[0], 3, 100, 100)
# TEST_IMG = TEST_IMG.reshape(TEST_IMG.shape[0], 3, 100, 100)
TRAIN_IMG = TRAIN_IMG.astype('float32')
TEST_IMG = TEST_IMG.astype('float32')
TRAIN_IMG /= 255
TEST_IMG /= 255

# Reshape class labels
TRAIN_CLS = np_utils.to_categorical(TRAIN_CLS, 10)
TEST_CLS = np_utils.to_categorical(TEST_CLS, 10)

# Shuffle the data
shuffle_data(TRAIN_IMG, TRAIN_CLS)
shuffle_data(TEST_IMG, TEST_CLS)

In [ ]:
for i in range(run_times):
    print("running" + str(i))
    for test_kernel in kernels:
        print("COMPILING SIMPLE MODEL")
        simple_model = Sequential()
        simple_model.add(Conv2D(num_filters, test_kernel, input_shape=input_shape))
        simple_model.add(Activation('relu'))
        simple_model.add(MaxPooling2D(pool_size=(2,2)))
        simple_model.add(Flatten()) # Converts and connects multi-dimentional convolutional layer into a 1D feature vector to be used for final classification
        simple_model.add(Dense(10, activation='softmax'))
        simple_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        print("TRAINING SIMPLE MODEL FOR {} EPOCHS using {} kernel size".format(EPOCHS, test_kernel)) 
        history = simple_model.fit(TRAIN_IMG, TRAIN_CLS, batch_size=32, epochs=EPOCHS, verbose=0, validation_split=0.2, shuffle=True)
        loss, acc = simple_model.evaluate(TEST_IMG, TEST_CLS, verbose=1)
        print("Loss: ", loss, " Accuracy: ", acc)
        with open("results_simple_cnn.txt", 'a') as f:
            f.write('{{ "accuracy":{}, "epochs":{}, "kernel":"{}", "num_filters":{}, "input_shape":"{}" }} \n'.format(
                acc, EPOCHS, test_kernel, num_filters, input_shape))

running0
COMPILING SIMPLE MODEL
TRAINING SIMPLE MODEL FOR 10 EPOCHS using (1, 1) kernel size
272/272 [==============================] - 4s 16ms/step
Loss:  14.458879414726706  Accuracy:  0.10294117647058823
COMPILING SIMPLE MODEL
TRAINING SIMPLE MODEL FOR 10 EPOCHS using (2, 2) kernel size
272/272 [==============================] - 5s 18ms/step
Loss:  14.518137539134306  Accuracy:  0.09926470588235294
COMPILING SIMPLE MODEL
TRAINING SIMPLE MODEL FOR 10 EPOCHS using (3, 3) kernel size
272/272 [==============================] - 5s 20ms/step
Loss:  1.3930527041940128  Accuracy:  0.6066176470588235
COMPILING SIMPLE MODEL
TRAINING SIMPLE MODEL FOR 10 EPOCHS using (4, 4) kernel size
272/272 [==============================] - 6s 22ms/step
Loss:  1.3333811900194954  Accuracy:  0.6433823529411765
COMPILING SIMPLE MODEL
TRAINING SIMPLE MODEL FOR 10 EPOCHS using (5, 5) kernel size
272/272 [==============================] - 7s 27ms/step
Loss:  14.458879302529727  Accuracy:  0.10294117647058823
COM